In [161]:
import csv
import datetime
import operator
import os
import re
from random import randint
import networkx as nx
import csv
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from operator import itemgetter
from shapely.wkt import loads
import copy
from os import path
import netwulf as nw
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mtick
from os import listdir
import time

In [162]:
NOT_INFECTED=0
INFECTED=1
NUM_NODOS=1
PERCENTAGE = 0.60

enlaces_global = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_GrafoGlobal.csv"
enlaces_duero = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO DUERO.csv"
enlaces_tajo = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO TAJO.csv"
enlaces_guadiana = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GUADIANA.csv"
enlaces_jucar = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO JUCAR.csv"
enlaces_ebro = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO EBRO.csv"
enlaces_segura = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO SEGURA.csv"
enlaces_garona = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GARONA.csv"
enlaces_barbate = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO BARBATE.csv"
enlaces_ter =  "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIU TER.csv"
enlaces_palancia = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIU PALANCIA.csv"
enlaces_guadalquivir = "/Users/silviadelatorre/Desktop/TFG/EDGE LIST/3 COORDS/Edgelist_20240311_125307_Grafo_RIO GUADALQUIVIR.csv"
rios = [(enlaces_guadalquivir,"RIO GUADALQUIVIR"),(enlaces_palancia,"RIO PALANCIA"),(enlaces_ter,"RIO TER"),(enlaces_barbate,"RIO BARBATE"),(enlaces_garona,"RIO GARONA"),(enlaces_segura,"RIO SEGURA"),(enlaces_guadiana,"RIO GUADIANA"),(enlaces_ebro,"RIO EBRO"),(enlaces_jucar,"RIO JUCAR"),(enlaces_tajo,"RIO TAJO")]

fosfato = "Results/VALORES CONTAMINACION/Fosfato_merged.csv"
amonio = "Results/VALORES CONTAMINACION/Amonio_merged.csv"
fosforo = "Results/VALORES CONTAMINACION/Fosforo_merged.csv"
fitobentos = "Results/VALORES CONTAMINACION/Fitobentos_merged.csv"
nitrato = "Results/VALORES CONTAMINACION/Nitrato_merged.csv"
grado_trofico = "Results/VALORES CONTAMINACION/Grado Trofico_merged.csv"


In [163]:

def CrearGrafo(edge_list):
    # Crear un grafo no dirigido
    G = nx.DiGraph()

    # Analizar el edge list y agregar los nodos y las aristas al grafo
    with open(edge_list, 'r') as f:
        for line in f:
            nodos_y_atributos = line.strip().split(';')
            nodo1_coords = tuple(map(float, nodos_y_atributos[0][1:-1].split(', ')))
            nodo2_coords = tuple(map(float, nodos_y_atributos[1][1:-1].split(', ')))

            # Agregar la arista al grafo
            G.add_edge(nodo2_coords, nodo1_coords)

    print("Número de nodos:", G.number_of_nodes())
    print("Número de aristas:", G.number_of_edges())
    degrees = G.degree()
    nodes = G.nodes()
    n_color = np.asarray([degrees[n] for n in nodes])
    #GraficarRed(G,n_color)
    return G

In [164]:

def StatusNodes(V):
    'Inicializar dataframe de nodos y estados de infeccion'
    global df, df
    
    Nodes=V.nodes()
    df = pd.DataFrame(Nodes)
    #print(df)
    data = {'Node': Nodes, 'Status': [NOT_INFECTED] * len(Nodes)}
    df = pd.DataFrame(data)

    #print(f'Status data frame: {df}')

In [165]:
def Infected_Nodes(V):
    global df

    #StatusNodes(V)
    TotalNodes = len(V.nodes())
    TotalNodesInfect = df['Status'].sum()

    Rate = TotalNodesInfect/TotalNodes
    return Rate


In [166]:
def CodigodelRio(rio):
    if(rio=='RIO GUADALQUIVIR'):
        codigo = '10094'
    elif(rio=='RIO GARONA'):
        codigo = '10034'
    elif(rio=='RIO MIÑO'):
        codigo = '10038'
    elif(rio=='RIO DUERO'):
        codigo = '1004'
    elif(rio=='RIO TAJO'):
        codigo = '1006'
    elif(rio=='RIO GUADIANA'):
        codigo = '1008'
    elif(rio=='RIO BARBATE'):
        codigo = '10098'
    elif(rio=='RI0 TER'):
        codigo = '20054'
    elif(rio=='RIO LLOBREGAT'):
        codigo = '20052'
    elif(rio=='RIO PALANCIA'):
        codigo = '20036'
    elif(rio=='RIO SEGURA'):
        codigo = '20016'
    elif(rio=='RIO EBRO'):
        codigo='2004'
    elif(rio=='RIO JUCAR'):
        codigo='2002'
    
    return codigo

In [167]:
def ObtenerPrimerNodo(G,agente,rio):
    max = 0
    coord_max_fosfato = None
    print(f'Obtener Nodo más contaminado del {rio}')
    codigo = CodigodelRio(rio)
    print(f"El codigo del {rio} es {codigo}")
    with open(agente, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            # Obtiene el ID del río, la coordenada y el valor de fosfato
            id_rio = row[0]
            coord = row[1]
            valor_contaminante = float(row[2])
            # Comprueba si el valor actual del agente es mayor que el máximo
            if str(id_rio).startswith(str(codigo)) and valor_contaminante > max:
                max = valor_contaminante
                coord_max = coord
                
    print("Mayor valor de contaminación es",max)
    return coord_max

In [168]:
def DibujarVariasGraficas(Dir,pC, pR):
    
    #mi_path = os.getcwd() + "\\" + "..\\datos\\" + str(Dir)
    mi_path=Dir
    
    fig = plt.figure()
    ax = plt.subplot(111)
    plt.title('Failure propagation')
    ax.set_xlabel('n')
    ax.set_ylabel('Infection rate')
    fmt = '%.5f' # Format you want the ticks, e.g. '40%'
    xticks = mtick.FormatStrFormatter(fmt)
    ax.yaxis.set_major_formatter(xticks)

    datosHeader = ['iter', 'rate']
    for NomFich in listdir(mi_path):
        pathFile = mi_path + "\\" + NomFich
        datos = pd.read_table(pathFile, 'engine=python', delimiter=' ', header=0, encoding = "ISO-8859-1", names=datosHeader)
        ax.plot(datos['iter'], datos['rate'], marker='o', label='pI: '+str(pC) + ' pR: '+str(pR))
        ax.legend()
    fileFig="TodosGraficos.png"
    pathFileFig = mi_path + "\\" + fileFig
    fig.savefig(pathFileFig)

In [169]:
def ComprobarInfeccion (N, p):
    
    if (np.random.binomial(N,p) > 0):
        return INFECTED
    else:
        return NOT_INFECTED

def ComprobarRecuperacion (p):
    
    if (np.random.binomial(1,p) > 0):
        return NOT_INFECTED
    else:
        return INFECTED 

In [170]:
def VerificarInfeccionVecinos(V,i):

    global df,pC
    
    Neighbours= V.neighbors(i)
    NumNodosInfect= df.Status[(df['Node'].isin(Neighbours)) & (df['Status']==INFECTED)].sum()
    df.Status[df.Node==i] = ComprobarInfeccion (NumNodosInfect, pC)

def AplicarRecuperacion(i):

    global df,pR
	
    df.status[(df['node']==i)]=ComprobarRecuperacion(pR)

In [171]:
def EjecutarPropagacionInfeccionTime(V,PERCENTAGE,Time):
    global Rate, TotalNodes, df
    
    pathFile = "Results/PROPAGACIÓN/ResultadosPropagación.csv"
    print("Time " + str(Time))

    #while Rate < PERCENTAGE:
    print("Propagar infección a vecinos")
    [VerificarInfeccionVecinos(V,i) for i in df.Node[df.Status == NOT_INFECTED]]
    TotalNodesInfect = df.Status[df.Status == INFECTED].sum()

    # Aplicar recuperación
    [AplicarRecuperacion(i) for i in df.Node[df.Status == INFECTED]]

    Rate = TotalNodesInfect / TotalNodes

    # Crear un DataFrame con los datos de tiempo y tasa de infección
    datos = pd.DataFrame([[Time, Rate]], columns=['time', 'rate'])
    '''
    DatosTime = pd.DataFrame([[0, 0]], columns=['time', 'rate'], dtype=int)
    DatosTime['time'] = Time
    DatosTime['rate'] = Rate'''

    datos.to_csv(pathFile, header=None, index=None, mode='a', sep=' ', decimal='.')

    DatosTime = DatosTime.drop(labels="time", axis=1)
    DatosTime = DatosTime.drop(labels="rate", axis=1)

    Time += 1
    #
    print("Infección alcanzó el " + str(PERCENTAGE * 100) + "% de los nodos INFECTEDs.")
    print("End time " + str(Time))

In [172]:
def EjecutarIteracionInfeccion(V,Tipo,nombre):
    '''Función para ejecutar una iteración de la simulación'''

    global pathFile, Time, df, Rate, TotalNodes,NDir
    
    Iter=0
    print("Iteracion " + str(Iter))
    
    print("Propagando infeccion")

    Nodes=V.nodes()
    TotalNodes = len(Nodes)
    #print("Total nodes " + str(TotalNodes))
    FirstNode = ObtenerPrimerNodo (V, Tipo,nombre)
    print("Primer nodo infectado")
    print(FirstNode)
    FirstNode = tuple(map(float, FirstNode.strip('()').split(', ')))
    df.loc[df['Node'] == FirstNode, 'Status'] = INFECTED

    df.to_csv('nodos_dataframe.csv', index=False)
    
    TotalNodesInfect = df.Status[df.Status==INFECTED].sum()
    print("Total nodos infectados:",TotalNodesInfect)
    
    Rate = Infected_Nodes(V)
    print(f"Ratio infectados INICIAL: {Rate * 100:.4f}%")

    # pathFile=pathFile.replace('.txt', '')
    # pathFile= pathFile + ".txt"
    #print("Directorio"+str(pathFile))
    #if (path.exists(pathFile)):
    #    os.remove(pathFile)
	
    start_time = time.time()

    while(Rate < PERCENTAGE):
        Iter += 1
        Rate = InfectarVecinos(V,FirstNode)
        #EjecutarPropagacionInfeccionTime(V,Rate,start_time)
        print("Ratio:",Rate)
    
    end_time = time.time()  # Registra el tiempo de finalización
    execution_time = end_time - start_time  # Calcula el tiempo de ejecución

    print(f"Ratio FINAL nodos infectados: {Rate * 100:.4f}%")
    print("End time "+str(execution_time))

    

In [ ]:
def CrearDirectorio(Tipo,rio):
    print("Creando directorio")
    NDir= "/Users/silviadelatorre/Desktop/TFG/PFG/Results/PROPAGACIÓN/Prop"+Tipo+"/"+rio
    if (path.exists(NDir)):
        print("")
    else:
        #shutil.rmtree(NDir)
        os.mkdir(NDir)

    #return Ndir

In [173]:

def PropagarInfeccion2 (pR, pC, Tipo,V,rio):

    global NDir
    
    print("Ejecutando iteraciones")
    
    if (Tipo == 'Am'):
        CrearDirectorio('Amonio',rio)
        EjecutarIteracionInfeccion(V,amonio,rio)
    elif (Tipo == 'Ni'):
        CrearDirectorio('Nitrato',rio)
        EjecutarIteracionInfeccion(V,nitrato,rio)
    elif (Tipo == 'Fr'):
        CrearDirectorio('Fosforo',rio)
        EjecutarIteracionInfeccion(V,fosforo,rio)
    elif (Tipo == 'Ft'):
        CrearDirectorio('Fosfato',rio)
        EjecutarIteracionInfeccion(V,fosfato,rio)
    elif (Tipo == 'Gt'):
        CrearDirectorio('GradoTrofico',rio)
        EjecutarIteracionInfeccion(V,grado_trofico,rio)
    else:
        CrearDirectorio('Fitobentos',rio)
        EjecutarIteracionInfeccion(V,fitobentos,rio)


In [174]:

def PropagarInfeccionTodasProbabilidades(i, Tipo,V,rio):
    global pC, pR
    
    pC=i
    pR=0
    print("Probabilidad contagio " + str(pC))

    PropagarInfeccion2 (pR, pC, Tipo,V,rio)

In [175]:
def Menu2(graph,nombre_rio):
    while True:
        print("Choose an option:")
        print("1. Propagation with infection starting in node with highest nitrate contamination")
        print("2. Propagation with infection starting in node with highest phosphorus contamination")
        print("3. Propagation with infection starting in node with highest phosphate contamination")
        print("4. Propagation with infection starting in node with highest ammonium contamination")
        print("5. Propagation with infection starting in node with highest phytobenthos contamination")
        print("6. Propagation with infection starting in node with highest trophic grade contamination")
        print("7. Propagation with infection starting in node with highest overall contamination")
        print("8. Exit")
        
        StatusNodes(graph)

        option = input("")

        if option == "1":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Nt",graph,nombre_rio)
            break
        elif option == "2":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Fr",graph,nombre_rio)
            break
        elif option == "3":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Ft",graph,nombre_rio)
            break
        elif option == "4":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Am",graph,nombre_rio)
            break
        elif option == "5":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Fit",graph,nombre_rio)
            break
        elif option == "6":
            rate = Infected_Nodes(graph)
            if(rate < PERCENTAGE): #and not all(state == "I"):
                PropagarInfeccionTodasProbabilidades(0.5,"Gt",graph,nombre_rio)
            break
        elif option == "7":
            break
        else:
            print("Invalid option. Please choose again.")

    return option

In [176]:

def Menu1():
    while True:
        print("Choose an option:")
        print("1. Simulate propagation for the global network of rivers")
        print("2. Simulate propagation for each individual non-connected subnetwork")
        print("3. Exit")

        option = input("")
        print("You chose option: ",option)

        if option == "1":
            print("Simulating propagation for the entire network, infecting one node per subnetwork")
            G = CrearGrafo(enlaces_global)
            Menu2(G,"RIOS ESPAÑA")
            return option
        
        elif option == "2":
            print("Simulating propagation for each individual subnetwork")
            
            for enlaces,nombre_rio in rios:
                print(f"Propagar infección en rio {nombre_rio}")
                G = CrearGrafo(enlaces)
                Menu2(G,nombre_rio)
            
            return option
        elif option == "3":
            print("Exiting")
            return option
        else:
            print("Invalid option. Please choose again.")



In [177]:
option1 = Menu1()

Choose an option:
1. Simulate propagation for the global network of rivers
2. Simulate propagation for each individual non-connected subnetwork
3. Exit
You chose option:  2
Simulating propagation for each individual subnetwork
Propagar infección en rio RIO GUADALQUIVIR
Número de nodos: 63815
Número de aristas: 66383
Choose an option:
1. Propagation with infection starting in node with highest nitrate contamination
2. Propagation with infection starting in node with highest phosphorus contamination
3. Propagation with infection starting in node with highest phosphate contamination
4. Propagation with infection starting in node with highest ammonium contamination
5. Propagation with infection starting in node with highest phytobenthos contamination
6. Propagation with infection starting in node with highest trophic grade contamination
7. Propagation with infection starting in node with highest overall contamination
8. Exit
Probabilidad contagio 0.5
Creando directorio

Ejecutando iteracio

/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/ipykernel_16566/268532437.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.Status[df.Node==i] = ComprobarInfeccion (NumNodosInfect, pC)
/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/

VECINOS:
(201553.4979, 4077563.3432)
(238035.2782, 4086690.7185)
(237934.0847, 4087941.5143)
VECINOS:
(201553.4979, 4077563.3432)
(267017.7907, 4098887.3068)
(253035.2668, 4111878.4869)
VECINOS:
(201553.4979, 4077563.3432)
(247929.5036, 4112211.2006)
(248453.739, 4118003.6345)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
(241440.7513, 4152266.39)
(251395.5723, 4153383.3429)
VECINOS:
(201553.4979, 4077563.3432)
(234835.0882, 4166967.267)
(234735.889, 4162647.9181)
VECINOS:
(201553.4979, 4077563.3432)
(252381.8374, 4158510.1496)
(256035.3065, 4159673.6102)
VECINOS:
(201553.4979, 4077563.3432)
(236680.1682, 4179186.324)
(235676.3259, 4179432.7496)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
(270390.0222, 4183277.0103)
(269567.6498, 4184147.7607)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
(287999.1084, 4163685.3368)
(284767.2429

/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/ipykernel_16566/268532437.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.Status[df.Node==i] = ComprobarInfeccion (NumNodosInfect, pC)
/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/

VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
(443945.0322, 4196733.8233)
(445794.6458, 4195040.9793)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
(453553.1933, 4196737.3237)
(454648.776, 4195382.925)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979, 4077563.3432)
(463487.5675, 4204416.383)
(462748.3706, 4202800.4671)
VECINOS:
(201553.4979, 4077563.3432)
VECINOS:
(201553.4979

...

VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
V

/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/ipykernel_16566/268532437.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.Status[df.Node==i] = ComprobarInfeccion (NumNodosInfect, pC)
/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/

VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(247201.4801, 4163507.8999)
VECINOS:
(230452.0428, 4203257.8787)
(230545.5304, 4204474.2275)
(230497.8245, 4204924.8539)
VECINOS:
(230452.0428, 4203257.8787)
VECINOS:
(230452.0428, 4203257.8787)
VECINOS:
(260139.6124, 4217620.0047)
VECINOS:
(252981.1165, 4212053.5603)
VECINOS:
(220677.3726, 4217757.8752)
VECINOS:
(228455.596, 4216104.9977)
VECINOS:
(220677.3726, 4217757.8752)
VECINOS:
(220677.3726, 4217757.8752)
VECINOS:
(220677.3726, 4217757.8752)
VECINOS:
(220677.3726, 4217757.8752)
VECINOS:
(220677.3726, 4217757.8752)
VECINOS:
(228952.5929, 4238307.6241)
VECINOS:
(228814.5025, 4232922.7446)
VECINOS:
(228952.592

/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/ipykernel_16566/268532437.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.Status[df.Node==i] = ComprobarInfeccion (NumNodosInfect, pC)
/var/folders/b9/7yv8wf7j1xd39p1ghzll0fqh0000gn/T/

KeyboardInterrupt: 